<a href="https://colab.research.google.com/github/Jeansds/Curso_Udemy---Deep-Learning-com-Python-de-A-a-Z/blob/master/Pr%C3%A9_Processamento_Projeto_Carros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# link download base dados
[link text](https://storage.googleapis.com/kaggle-data-sets/402/793065/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1583407591&Signature=TEsbAF4fzLr2smCA94imiqVJjTfaXC3Dn4aEBaOVzIahvI0p3FIpWKAzIArV%2FTbQRsUGCi1Uc0uWhe3hLaFVIsbiFIB%2FUeml8xexM73oEo%2BZndGOcZrz9jq81ssg%2FC0WbI3Piopg1IB6chmfFhQw352d1Sa5eyHKIa2p4sA4xI6N6k0nN1CvOMUwIm7jtveHWGoqdPlZbvH4emV0dLKbqtGQ%2FzThoPPLqC3s%2BmyoHCFf%2BL7nxkcke3j7gaaqlSRE7Q3SMZC66apPXJrwMTh5ZwM5to3g08X%2BlDv44nRn6VcqUoj6T%2BIJwyEfL4DSftGCq2xg15SeATsYu2%2FWQfX6CQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dused-cars-database.zip)

In [0]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import cross_val_score
from keras.wrapper.scikit-learn import KerasRegressor

In [45]:
base = pd.read_csv('/content/sample_data/autos.csv', encoding = 'ISO-8859-1')
Dados = ['name', 'seller', 'offerType']
print([base[i].value_counts() for i in Dados])

[Ford_Fiesta                                       657
BMW_318i                                          627
Opel_Corsa                                        622
Volkswagen_Golf_1.4                               603
BMW_316i                                          523
                                                 ... 
Volkswagen_Fox_1.2_schwarz_Autogas                  1
Peugeot_206_cc_Platinium_136Ps                      1
VW_POLO_6n2_Kombi                                   1
Renault_Twingo_mit_TÜV___NEU!!!                     1
Opel_Astra_Cabrio_1.6i_Bertone__LEDER__el.Dach      1
Name: name, Length: 233531, dtype: int64, privat        371525
gewerblich         3
Name: seller, dtype: int64, Angebot    371516
Gesuch         12
Name: offerType, dtype: int64]


In [0]:
Dados=['dateCrawled', 'dateCreated', 'nrOfPictures', 'postalCode', 'lastSeen', 'name', 'seller', 'offerType']
for i in Dados:
  base = base.drop(i, axis = 1)

In [47]:
print(base.loc[base.price <= 10])
base = base[base.price > 10]
print(base.loc[base.price > 350000])
base = base.loc[base.price < 350000]

        price   abtest vehicleType  ...  fuelType           brand  notRepairedDamage
7           0     test   limousine  ...    benzin      volkswagen               nein
40          0     test         NaN  ...    benzin            opel                NaN
60          1  control         suv  ...       NaN  sonstige_autos                NaN
91          1  control   limousine  ...    diesel   mercedes_benz               nein
115         0     test         NaN  ...    benzin      volkswagen                NaN
...       ...      ...         ...  ...       ...             ...                ...
371356      0  control         NaN  ...       NaN            opel                 ja
371392      0     test  kleinwagen  ...    benzin            ford                NaN
371402      0  control  kleinwagen  ...    benzin          suzuki                NaN
371431      0  control  kleinwagen  ...    benzin            seat                 ja
371522      0  control         NaN  ...    benzin      mitsubishi

In [48]:
Dados = ['vehicleType', 'gearbox', 'model', 'fuelType', 'notRepairedDamage']
for i in Dados:
  print(base.loc[pd.isnull(base[i])])
  print(base[i].value_counts())

        price   abtest vehicleType  ...  fuelType           brand  notRepairedDamage
0         480     test         NaN  ...    benzin      volkswagen                NaN
16        300     test         NaN  ...    benzin      volkswagen                NaN
22       2900     test         NaN  ...    benzin            opel               nein
26       5555  control         NaN  ...       NaN         citroen               nein
31        899  control         NaN  ...    benzin         renault                NaN
...       ...      ...         ...  ...       ...             ...                ...
371495    180  control         NaN  ...    benzin            opel                NaN
371504   2600  control         NaN  ...       NaN   mercedes_benz                NaN
371509   1900     test         NaN  ...       NaN      volkswagen               nein
371519   5250  control         NaN  ...       NaN      alfa_romeo               nein
371523   2200     test         NaN  ...       NaN  sonstige_autos

In [0]:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

In [50]:
previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values
Transformadores = [0, 1, 3, 5, 8, 9, 10]
labelenconder_previsores = LabelEncoder()
for i in Transformadores:
  previsores[:, i] = labelenconder_previsores.fit_transform(previsores[:, i])
print(previsores[0])

[1 6 1993 1 0 118 150000 0 1 38 1]


In [51]:
onehotencorder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), Transformadores)],remainder='passthrough')
previsores = onehotencorder.fit_transform(previsores).toarray()
print(previsores)

[[0.00e+00 1.00e+00 0.00e+00 ... 0.00e+00 1.50e+05 0.00e+00]
 [0.00e+00 1.00e+00 0.00e+00 ... 1.90e+02 1.25e+05 5.00e+00]
 [0.00e+00 1.00e+00 0.00e+00 ... 1.63e+02 1.25e+05 8.00e+00]
 ...
 [0.00e+00 1.00e+00 0.00e+00 ... 1.02e+02 1.50e+05 3.00e+00]
 [0.00e+00 1.00e+00 0.00e+00 ... 1.00e+02 1.50e+05 6.00e+00]
 [1.00e+00 0.00e+00 0.00e+00 ... 3.20e+02 5.00e+04 8.00e+00]]


In [53]:
#Inicio da Construção Rede Neural
regressor = Sequential()
regressor.add(Dense(units = 158, activation = 'relu', input_dim = 316))
regressor.add(Dense(units = 158, activation = 'relu'))
regressor.add(Dense(units = 1, activation = 'linear'))
regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam',
                  metrics = ['mean_absolute_error'])
regressor.fit(previsores, preco_real, batch_size = 300, epochs = 100)

Epoch 1/100





359291/359291 [==============================] - 10s 28us/step - loss: 3951.4921 - mean_absolute_error: 3951.4921
Epoch 2/100
359291/359291 [==============================] - 9s 26us/step - loss: 3451.4364 - mean_absolute_error: 3451.4364
Epoch 3/100
359291/359291 [==============================] - 9s 26us/step - loss: 3347.2336 - mean_absolute_error: 3347.2336
Epoch 4/100
359291/359291 [==============================] - 9s 26us/step - loss: 3169.5397 - mean_absolute_error: 3169.5397
Epoch 5/100
359291/359291 [==============================] - 9s 26us/step - loss: 2991.0964 - mean_absolute_error: 2991.0964
Epoch 6/100
359291/359291 [==============================] - 9s 26us/step - loss: 2892.5311 - mean_absolute_error: 2892.5311
Epoch 7/100
359291/359291 [==============================] - 10s 27us/step - loss: 2809.7109 - mean_absolute_error: 2809.7109
Epoch 8/100
359291/359291 [==============================] - 9s 26us/step - loss: 2752.1980 - mean_absolute_error: 275

In [0]:
def criar_rede():
  regressor = Sequential()
  regressor.add(Dense(units = 158, activation = 'relu', input_dim = 316))
  regressor.add(Dense(units = 158, activation = 'relu'))
  regressor.add(Dense(units = 1, activation = 'linear'))
  regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam',
                    metrics = ['mean_absolute_error'])
  return regressor

In [0]:
#validação cruzada
regressor = KerasRegressor(build_fn = criar_rede,
                           epochs = 100,
                           batch_size = 300)
resultados = cross_val_score(estimator = regressor,
                             X = previsores, y = preco_real,
                             cv = 10, scoring = 'mean_absolute_error')
print(media = resultados.mean())
print(desvio = resultados.std())